In [ ]:
# Допустимые значения: москва, новосибирск
city = input("Введите название города: ").lower()

Введите название города: москва


In [ ]:
! pip3 install google-api-python-client google-auth-httplib2 google-auth-oauthlib tabulate requests tqdm
! pip install pydrive
# ! pip install pypdfium2==2.0.0
! pip install PyPDF2
# !pip install --upgrade openpyxl
! pip install gspread --user
! pip install gspread-formatting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.3 MB/s eta 0:00:00


In [ ]:
import os, glob, csv, re, io, time, requests
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from tqdm import tqdm
import pandas as pd
from google.colab import files
pd.options.mode.chained_assignment = None  # default='warn'
# import pypdfium2 as pdfium
# from PIL import Image
# import openpyxl
# from openpyxl.styles.alignment import Alignment
# from openpyxl import load_workbook
# from openpyxl.drawing.xdr import XDRPoint2D, XDRPositiveSize2D
# from openpyxl.styles import Alignment
# from openpyxl.styles import Font
# from openpyxl.styles import Border, Side
# from openpyxl.drawing.image import Image as openpyxl_image
from PyPDF2 import PdfMerger
from google.auth.transport.requests import AuthorizedSession
# from io import BytesIO
# warnings.simplefilter('ignore')
# import gc

import gspread
from gspread_formatting  import *
from gspread import authorize
from zipfile import ZipFile
from urllib.request import urlopen
from io import StringIO

import numpy as np

In [ ]:
from google.colab import drive
drive.mount('drive')

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
google_drive = GoogleDrive(gauth)

from googleapiclient.discovery import build
from google.colab import auth
from google.auth import default

# auth.authenticate_user()
creds, _ = default()

# Build the services
service_drive = build('drive', 'v3', credentials=creds)
service_sheets = build('sheets', 'v4', credentials=creds)

Mounted at drive


In [ ]:
gclient = gspread.authorize(creds)

In [ ]:
translit = {'a' : 'а', 'b' : 'б', 'c' : 'ц', 'd' : 'д', 'e' : 'е', 'f' : 'ф', 'g' : 'г', 'h' : 'х', 'i' : 'и', 'j' : 'й', 'k' : 'к', 'l' : 'л', 'm' : 'м', 'n' : 'н', 'o' : 'о', 'p' : 'п', 'r' : 'р', 's' : 'с', 't' : 'т', 'u' : 'у', 'v' : 'в', 'w' : 'вв', 'x' : 'кс', 'y' : 'ы', 'z' : 'з'}

In [ ]:
nums = {'1' : 'один', '2' : 'два', '3' : 'три', '4' : 'четыре', '5' : 'пять', '6' : 'шесть', '7' : 'семь', '8' : 'восемь', '9' : 'девять', '0' : 'ноль'}

In [ ]:
mergers = ['-', ' ']

In [ ]:
# black_list = ['00300', '00213']
# black_list = [00300, 00213]
# '00213' in black_list

In [ ]:
# codeword = 'гусь'

In [ ]:
if city == 'москва':
  codeword = 'гусь'
  submissions_path = "/content/drive/My Drive/2024/москва/файлы работ/"
  pdf_parent_folder = '1PzfV-Gy9xTVfz6fl_4sWQIFnbQoCYpp8' # папка с файлами работ
  tables_path = "/content/drive/My Drive/2024/москва/технические файлы/таблицы xlsx/" # Не нужна, можно удалить
  spreadsheet_path = "1BzuCYho2YyPvz3xxx2DMYVHdS4eMMqwu" # Папка с гугл-таблицами учета по площадкам
  log_sheet_url = "https://docs.google.com/spreadsheets/d/1PNK4su_Ie7IQBM-QAx1laKr1XqVjD3cqfObTCQAJGR0/edit#gid=1763744115" #Сводная таблица
  log_url = log_sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
  sites_folder = '1g4QtUegF2X5mc60lhu22ucqgYafAWDxd'# Папка файлы площадок (кажется, она здесь не нужна)
  final_path = '1KZhKnREeVPIwFzfAHLX3w5D_oaQVOOv7'# id папки загрузка на сайт
  final_dest = "/content/drive/My Drive/2024/москва/загрузка на сайт"
  log_id = '1PNK4su_Ie7IQBM-QAx1laKr1XqVjD3cqfObTCQAJGR0' # id Сводной таблицы

# if city == 'новосибирск':
  # codeword = 'новосибирск'
  # submissions_path = "/content/drive/My Drive/2024/новосибирск/файлы работ/"
  # pdf_parent_folder = '1h0o8nzy0gH1BCOmpI4SAbomiGL1SGFLG' # Папка файлы_работ
  # tables_path = "/content/drive/My Drive/2024/новосибирск/технические файлы/таблицы xlsx/" # Не нужна, можно удалить
  # spreadsheet_path = "1lhO1ILwBwkNQSweo53KWLi49-I0qJjJM" # Папка таблицы учета
  # log_sheet_url = "https://docs.google.com/spreadsheets/d/17z17A8eqCqtMjnfRI2gK8k-hHCX_670guZEbG9pbZ9o/edit#gid=1763744115" # Сводная таблица
  # log_url = log_sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
  # sites_folder = '1RpZOXd4wzxK5ZE7bWFd6EtXgcldgbu03'# Папка файлы_площадок
  # final_path = '15oI3QwmZo6Bb8lrg_5I4aMUnN8eVRk-w'# id папки загрузка на сайт
  # final_dest = "/content/drive/My Drive/2024/новосибирск/загрузка на сайт"
  # log_id = '17z17A8eqCqtMjnfRI2gK8k-hHCX_670guZEbG9pbZ9o'# id Сводной таблицы

  print(city)

  missing_grades = []

  statuses = ['Ожидает загрузки', 'На модерации', 'Загружено', 'Пропущено кол-во ошибок', 'csv ожидает повторной загрузки']

  global_log = pd.read_csv(log_url, converters={'№ площадки': str})
  # global_log = global_log.loc[(global_log['Статус'] == 'Готово') & (~global_log['Загрузка на сервер'].isin(statuses))]

  # special_request = ['72245', '72235']
  # global_log = global_log.loc[(global_log['Статус'] == 'Готово') & (~global_log['Загрузка на сервер'].isin(statuses)) & (global_log['№ площадки'].isin(special_request))]

  # global_log['№ площадки'] = global_log['№ площадки'].astype(str)
  global_log = global_log['№ площадки'].values

  print('Всего площадок для обработки: ', len(global_log))
  counter = 0

  for site in global_log:
    if site in ['72875']:
      # print(site)
    # #   # if (site == '50300'):
    # #       # print(site)
      log_wrkb = gclient.open_by_key(log_id)
      worksheet = log_wrkb.get_worksheet(0)
      cell = worksheet.find(str(site)) #Find a cell with exact string value
      # print('!!', cell)
      row, col = cell.row, cell.col


      print('processing site: ', str(site))
      print('Начало обработки: ', time.ctime(time.time()))
      # # final_folder = google_drive.CreateFile({'title': str(site), 'parents':[{'id':final_path}], 'mimeType': "application/vnd.google-apps.folder"})
      # # final_folder.Upload()
      # # print('final_folder created')

      # # Считываем вручную заполненные данные каждой работы из таблиц папки "таблицы учета"
      query = f"name contains '{site}' and trashed=false and parents='{spreadsheet_path}'"
      results = service_drive.files().list(q=query, fields='files(id, name, parents, trashed, createdTime)', orderBy='createdTime').execute()
      # print(results)
      table_id = results.get('files', [])[0]['id']
      # print(table_id)
      gsheet = gclient.open_by_key(table_id)
      # time.sleep(1)
      df = pd.DataFrame(gsheet.get_worksheet(0).get_all_records())
      # Если не указаны Фамилия, Имя или Кодовое слово -- заменяем их на дефолтное кодовое слово (см. переменная codeword)
      df[["Фамилия", "Имя", "Кодовое слово"]] = df[["Фамилия", "Имя", "Кодовое слово"]].fillna(value=codeword)
      df[["Фамилия", "Имя", "Кодовое слово"]] = df[["Фамилия", "Имя", "Кодовое слово"]].replace('', codeword, regex=True)

      # Проверяем, чтобы не было пропусков в данных
      cond_1 = df['Пункт. ошибки'].isnull().values.any()
      cond_2 = df['Орф. ошибки'].isnull().values.any()
      cond_3 = (df['Орф. ошибки'].eq('')).any()
      cond_4 = (df['Пункт. ошибки'].eq('')).any()

      if (cond_1 == True) or (cond_2 == True) or (cond_3 == True) or (cond_4 == True):
        # Если обнаружены пропуски в количестве ошибок, записывем в Сводную таблицу
        worksheet.update_acell(f'G{row}', 'Пропущено кол-во ошибок')
        missing_grades.append(site)
        continue
      else:
        df[['Фамилия', 'Имя', 'Кодовое слово', 'Орф. ошибки', 'Пункт. ошибки']] = df[['Фамилия', 'Имя', 'Кодовое слово', 'Орф. ошибки', 'Пункт. ошибки']].fillna('')
        df[['Фамилия', 'Имя', 'Кодовое слово']] = df[['Фамилия', 'Имя', 'Кодовое слово']].apply(lambda x: x.astype(str).str.lower())
        # Находим работы, состоящие из двух листов -- для последующего объединения
        duplicate = df[df.duplicated(subset=['Фамилия', 'Имя', 'Кодовое слово'], keep=False)]
        grouped = duplicate.groupby(['Фамилия', 'Имя', 'Кодовое слово'], sort=False)
        print("Работ для объединения pdf: ", len(grouped))

        # Объединяем pdf-ы работ, состоящих из двух листов, записываем объединенных pdf в папку "файлы работ", добавляем данные об объединенном файле в новый датафрейм.
        df_vals = []
        for name, group in grouped:
          merger = PdfMerger()
          parts = []
          for filename in group['Id работы'].values:
            path_1 = submissions_path + filename + '.pdf'
            merger.append(path_1)
            split_filename = filename.split('_')
            if split_filename[0] not in parts:
              parts.append(split_filename[0])
            parts.append(split_filename[1])
          new_filename = '_'.join(parts) + '.pdf'
          # print(new_filename)
          path_2 = submissions_path + new_filename
          merger.write(path_2)
          new_row = group.index[0], new_filename.strip('.pdf'), group['Фамилия'].values[0], group['Имя'].values[0], group['Кодовое слово'].values[0], group['Орф. ошибки'].values[0], group['Пункт. ошибки'].values[0]
          df_vals.append(new_row)

        duplicates_df = pd.DataFrame(df_vals, columns = ['index', 'Id работы', 'Фамилия', 'Имя', 'Кодовое слово', 'Орф. ошибки', 'Пункт. ошибки'])
        print('duplicates_df', duplicates_df.shape)

        # Добавляем в новый датафрейм данные об остальных работах (не требующих объединения)
        rest_df = df.loc[~df.index.isin(duplicate.index)]
        rest_df['index'] = rest_df.index
        rest_df = rest_df.iloc[:, [9, 0,2,3, 4, 6, 7]]
        rest_df.columns = duplicates_df.columns
        print('rest_df', rest_df.shape)

        new_df = pd.concat([duplicates_df, rest_df])
        new_df.sort_values(by=['index'], inplace=True)
        new_df.reset_index(inplace=True, drop=True)
        new_df = new_df.iloc[:, [1, 2, 3, 4, 5, 6]]
        new_df.columns = ['id', 'surname', 'name', 'code', 'orth', 'punct']
        print('new_df', new_df.shape)

        # В новом датафрейме: разбиваем строки, если в ячейках Фамилия, Имя или Кодовое слово проверяющими были перечислены несколько вариантов через запятую
        # Для каждого нового варианта создаем новую строку
        new_df = new_df.assign(surname=new_df['surname'].str.split(',')).explode('surname')
        new_df = new_df.assign(name=new_df['name'].str.split(',')).explode('name')
        new_df = new_df.assign(code=new_df['code'].str.split(',')).explode('code')
        # Транслитерируем
        for key, val in translit.items():
          new_df[["surname", "name", "code"]] = new_df[["surname", "name", "code"]].replace(key, val, regex=True)
        # Заменяем числительные на слова
        for key, val in nums.items():
          new_df[["surname", "name", "code"]] = new_df[["surname", "name", "code"]].replace(key, val, regex=True)
        # Удаляем дефисы и пробелы
        # for merger in mergers:
        #   new_df[["surname", "name", "code"]] = new_df[["surname", "name", "code"]].replace(merger, '', regex=True)
        # Удаляем любые другие неалфавитные символы
        new_df['surname'] = new_df['surname'].str.replace('[^ёа-яЁА-Я]', '', regex=True)
        new_df['name'] = new_df['name'].str.replace('[^ёа-яЁА-Я]', '', regex=True)
        new_df['code'] = new_df['code'].str.replace('[^ёа-яЁА-Я]', '', regex=True)
        new_df.reset_index(inplace=True, drop=True)
        new_df["ФИО"] = new_df["surname"] + ' ' + new_df["name"]
        new_df = new_df[['ФИО', 'orth', 'punct', 'code', 'id']]
        new_df['id'] = new_df['id'].astype(str) + '.pdf'
        new_df.columns = ['ФИО', 'Орфография', 'Пунктуация', 'Кодовое слово', 'Скан']
        print('new_df after modification', new_df.shape)

        query_1 = f"name = '{site}' and trashed=false and parents='{pdf_parent_folder}'and mimeType='application/vnd.google-apps.folder'"
        results_1 = service_drive.files().list(q=query_1, fields='files(id, name, parents, trashed, createdTime)', orderBy='createdTime').execute()
        folder = results_1.get('files', [])
        if len(folder) > 0:
          print('folder exists', folder)
          # print('folder exists: ', folder['name'])
          for f in folder:
            google_drive.CreateFile({'id': f['id']}).Trash()
            print('folder deleted')
            copy_folder = google_drive.CreateFile({'title': str(site), 'parents':[{'id':pdf_parent_folder}], 'mimeType': "application/vnd.google-apps.folder"})
            copy_folder.Upload()
            print('subfolder created')

          # break
        else:
          copy_folder = google_drive.CreateFile({'title': str(site), 'parents':[{'id':pdf_parent_folder}], 'mimeType': "application/vnd.google-apps.folder"})
          copy_folder.Upload()
          print('subfolder created')

        query_1 = f"name contains '{str(site)}' and trashed=false and parents='{pdf_parent_folder}' and mimeType='application/vnd.google-apps.folder'"
        copy_folder_1 = service_drive.files().list(q=query_1, fields='files(id, name, parents, trashed, createdTime)', orderBy='createdTime').execute()
        copy_folder_id = copy_folder_1.get('files', [])[0]['id']
        # print(copy_folder_id)

        new_list = list(reversed(new_df['Скан'].values.tolist()))
        # print(new_list)

        for id in tqdm(list(reversed(new_df['Скан'].values.tolist()))):
          # print(id)
          query = f"name contains '{id}' and trashed=false and parents='{pdf_parent_folder}'"
          # time.sleep(3)
          results = service_drive.files().list(q=query, fields='files(id, name, parents, trashed, createdTime)', orderBy='createdTime').execute()
          # print('RESULTS', results, '\n')
          interim = results.get('files', [])
          # print('INTERIM', interim)
          if len(interim) == 0:
            # print(len(results))
            print('SKIPPED: pdf not retrieved!!!')
            print('--------------------------------\n')
            break
          else:
            # print(len(results))
            pdf = interim[0]
            service_drive.files().copy(fileId=pdf['id'], body={"parents": [copy_folder_id], "mimeType":"application/pdf"} ).execute()
            # # print('pdf: ', pdf)
            # query_1 = f"name = '{pdf['name']}' and trashed=false and parents='{copy_folder_id}'and mimeType='application/pdf'"
            # results_1 = service_drive.files().list(q=query_1, fields='files(id, name, parents, trashed, createdTime)', orderBy='createdTime').execute()
            # copy = results_1.get('files', [])
            # print(copy)

        time.sleep(5)
        copy_files = google_drive.ListFile({'q': f"'{copy_folder_id}' in parents and trashed=false"}).GetList()
        print('saved copies: ', len(copy_files))

        if len(copy_files) != new_df.shape[0]:
          print('ERROR!')
          print('---------------------------\n')
          continue

        else:
          time.sleep(10)
          query_2 = f"name = '{site}' and trashed=false and parents='{final_path}'and mimeType='application/vnd.google-apps.folder'"
          results_2 = service_drive.files().list(q=query_2, fields='files(id, name, parents, trashed, createdTime)', orderBy='createdTime').execute()
          folder_2 = results_2.get('files', [])
          if len(folder_2) > 0:
            print('folder exists: ', folder)
            for f in folder_2:
              google_drive.CreateFile({'id': f['id']}).Trash()
              print('folder deleted')
              copy_folder = google_drive.CreateFile({'title': str(site), 'parents':[{'id':final_path}], 'mimeType': "application/vnd.google-apps.folder"})
              copy_folder.Upload()
              print('upload subfolder created')
              # time.sleep(50)
              for _ in tqdm(range(50),desc="waiting..."):
                time.sleep(1)
              query_3 = f"name = '{site}' and trashed=false and parents='{final_path}'and mimeType='application/vnd.google-apps.folder'"
              results_3 = service_drive.files().list(q=query_3, fields='files(id, name, parents, trashed, createdTime)', orderBy='createdTime').execute()
              folder_3 = results_3.get('files', [])
              print(len(folder_3))
              if len(folder_3) == 0:
                print('SKIPPED: final folder not retrived!!!')
                print('--------------------------\n')
                break
              # break
          else:
            copy_folder = google_drive.CreateFile({'title': str(site), 'parents':[{'id':final_path}], 'mimeType': "application/vnd.google-apps.folder"})
            copy_folder.Upload()
            print('upload subfolder created')
            # time.sleep(50)
            for _ in tqdm(range(50),desc="waiting..."):
              time.sleep(1)
            query_3 = f"name = '{site}' and trashed=false and parents='{final_path}'and mimeType='application/vnd.google-apps.folder'"
            results_3 = service_drive.files().list(q=query_3, fields='files(id, name, parents, trashed, createdTime)', orderBy='createdTime').execute()
            folder_3 = results_3.get('files', [])
            print(len(folder_3))
            if len(folder_3) == 0:
              print('SKIPPED: final folder not retrived!!!')
              print('--------------------------\n')
              break

          time.sleep(30) # Только для больших площадок. Убрать для маленьких!!!!!!!!!!!!!!!!!!
          isExist = os.path.exists(final_dest + '/' + str(site))
          print(isExist)
          if isExist == True:
            output_csv = final_dest + '/' + str(site) + '/' + str(site) + '.csv'
            print(output_csv)
            new_df.to_csv(output_csv, sep=';', encoding='cp1251', index=False)
            print('csv written')

            input_zip = submissions_path + str(site)
            print('input_zip', input_zip)
            # output_zip = final_dest + '/' + str(site) + '/' + str(site) + '.zip'
            # print('output_zip', output_zip)

            file_paths = []

            submissions_folder = submissions_path + str(site)

            for root, directories, files in os.walk(submissions_folder):
              for filename in files:
                # print(filename)
                file_path = os.path.join(root, filename)
                file_paths.append(file_path)


            output_zip = final_dest + '/' + str(site) + '/' + str(site) + '.zip'
            print('output_zip', output_zip)

            with ZipFile(output_zip, 'w') as zip:
              for file in tqdm(file_paths):
                # print(file.split('/')[-1])
                zip.write(file, file.split('/')[-1])
            print('zipfile written')

          else:
            print('SKIP: final folder not found\n')
            continue

          worksheet.update_acell(f'G{row}', 'Ожидает загрузки')
          counter+=1
          print('Обработано площадок: ', counter)
          print('-----------------------------------------\n')
          time.sleep(20)



  print('missing grades:', missing_grades)
  print('Обработка всех площадок завершена')

москва
Всего площадок для обработки:  211
processing site:  72875
Начало обработки:  Thu May 16 09:15:15 2024
Работ для объединения pdf:  0
duplicates_df (0, 7)
rest_df (22, 7)
new_df (22, 6)
new_df after modification (22, 5)
folder exists [{'parents': ['1PzfV-Gy9xTVfz6fl_4sWQIFnbQoCYpp8'], 'id': '1YJKGS-x5A3XPsLPWolPF0-aouz5RZU2T', 'name': '72875', 'trashed': False, 'createdTime': '2024-05-07T20:14:15.371Z'}]
folder deleted
subfolder created


100%|██████████| 22/22 [00:28<00:00,  1.30s/it]


saved copies:  22
folder exists:  [{'parents': ['1PzfV-Gy9xTVfz6fl_4sWQIFnbQoCYpp8'], 'id': '1YJKGS-x5A3XPsLPWolPF0-aouz5RZU2T', 'name': '72875', 'trashed': False, 'createdTime': '2024-05-07T20:14:15.371Z'}]
folder deleted
upload subfolder created


waiting...: 100%|██████████| 50/50 [00:50<00:00,  1.00s/it]


1
True
/content/drive/My Drive/2024/москва/загрузка на сайт/72875/72875.csv
csv written
input_zip /content/drive/My Drive/2024/москва/файлы работ/72875
output_zip /content/drive/My Drive/2024/москва/загрузка на сайт/72875/72875.zip


100%|██████████| 22/22 [00:06<00:00,  3.37it/s]


zipfile written
Обработано площадок:  1
-----------------------------------------

missing grades: []
Обработка всех площадок завершена


In [ ]:
# Новосибирск
# log_sheet_url = "https://docs.google.com/spreadsheets/d/1QRoeaXl2vuoVhUUEH-He6TQdb4L-501Tb-uR3ABql4o/edit#gid=1763744115" # Сводная таблица
# log_url = log_sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

# Москва
log_sheet_url = "https://docs.google.com/spreadsheets/d/1OlfzhB_XDwfbcTOVB3spWeDbGla2YEp9BlLq9wCIxP4/edit#gid=1763744115"
log_url = log_sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

statuses = ['Ожидает загрузки', 'На модерации', 'Загружено', 'Пропущены оценки', 'csv ожидает повторной загрузки']

global_log = pd.read_csv(log_url, converters={'№ площадки': str})
# global_log = global_log.loc[(global_log['Статус'] == 'Готово') & (global_log['Комментарий заполняющего (при необходимости)'].isin([np.nan])) & (~global_log['Загрузка на сервер'].isin(statuses))]
global_log = global_log.loc[(global_log['Статус'] == 'Готово') & (~global_log['Загрузка на сервер'].isin(statuses))]

print(global_log.shape)
global_log
# global_log = global_log['№ площадки'].values

(1, 8)


,№ площадки,Кол-во сканов,"Фамилия, имя заполняющего",Статус,Комментарий заполняющего (при необходимости),Название площадки,Загрузка на сервер,Unnamed: 7
232,68846,36,Бадрызлова Юлия,Готово,NaN,NaN,NaN,Нет на диске


In [ ]:
# Count folders
parent_folder = '1-4eEXVRv2CQtmPE4aOxefPUfHV7_eEgK'
query = f"trashed=false and parents='{parent_folder}' and mimeType='application/vnd.google-apps.folder'"

page_token = None
my_files = list()
while True:
  results = service_drive.files().list(pageSize=1000, pageToken=page_token, q=query, orderBy='createdTime').execute()
  files = results.get('files', [])
  my_files.extend(files)
  page_token = results.get('nextPageToken', None)
  if page_token is None:
      break

# lst = service_drive.files().list(q=query, fields='files(id, name, parents, trashed, createdTime)', orderBy='createdTime').execute()
# files = lst.get('files', [])

print(len(my_files))

# for file in my_files:
#   newTitle = file['name'][1:]
#   print(newTitle)
#   body = {'name': newTitle}
#   service_drive.files().update(fileId=file['id'], body=body).execute()

82


In [ ]:
# Rename / count files

site = '66870_'
# pdf_parent_folder = '1n1HBzvhgQzao7N368HVqKKgsLIKJ6uHw'

# Moscow
pdf_parent_folder = '10GWqMWoVAVgxvdcFF3D_x4iqj3OeVoYl'
query = f"name contains '{str(site)}' and trashed=false and parents='{pdf_parent_folder}' and mimeType='application/pdf'"

page_token = None
my_files = list()
while True:
  results = service_drive.files().list(pageSize=1000, pageToken=page_token, q=query, orderBy='createdTime').execute()
  files = results.get('files', [])
  my_files.extend(files)
  page_token = results.get('nextPageToken', None)
  if page_token is None:
      break

# lst = service_drive.files().list(q=query, fields='files(id, name, parents, trashed, createdTime)', orderBy='createdTime').execute()
# files = lst.get('files', [])

print(len(my_files))

for file in my_files:
  newTitle = file['name'][1:]
  print(newTitle)
  # body = {'name': newTitle}
  # service_drive.files().update(fileId=file['id'], body=body).execute()

63
6870_1.pdf
6870_2.pdf
6870_3.pdf
6870_4.pdf
6870_5.pdf
6870_6.pdf
6870_7.pdf
6870_8.pdf
6870_9.pdf
6870_10.pdf
6870_11.pdf
6870_12.pdf
6870_13.pdf
6870_14.pdf
6870_15.pdf
6870_16.pdf
6870_17.pdf
6870_18.pdf
6870_19.pdf
6870_20.pdf
6870_21.pdf
6870_22.pdf
6870_23.pdf
6870_24.pdf
6870_25.pdf
6870_26.pdf
6870_27.pdf
6870_28.pdf
6870_29.pdf
6870_30.pdf
6870_31.pdf
6870_32.pdf
6870_33.pdf
6870_34.pdf
6870_35.pdf
6870_36.pdf
6870_37.pdf
6870_38.pdf
6870_39.pdf
6870_40.pdf
6870_41.pdf
6870_42.pdf
6870_43.pdf
6870_44.pdf
6870_45.pdf
6870_46.pdf
6870_47.pdf
6870_48.pdf
6870_49.pdf
6870_50.pdf
6870_51.pdf
6870_52.pdf
6870_53.pdf
6870_54.pdf
6870_55.pdf
6870_56.pdf
6870_57.pdf
6870_58.pdf
6870_59.pdf
6870_60.pdf
6870_61.pdf
6870_29_30.pdf
6870_59_60.pdf


In [ ]:
for _ in tqdm(range(10),desc="waiting..."):
    time.sleep(1)

waiting...: 100%|██████████| 10/10 [00:10<00:00,  1.00s/it]


In [ ]:
log_sheet_url = "https://docs.google.com/spreadsheets/d/1OlfzhB_XDwfbcTOVB3spWeDbGla2YEp9BlLq9wCIxP4/edit#gid=1763744115"
log_url = log_sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

global_log = pd.read_csv(log_url)
global_log = global_log.loc[(global_log['Статус'] == 'Готово') & (global_log['Комментарий заполняющего (при необходимости)'].isin([np.nan])) & (global_log['Загрузка на сервер'] != 'Ожидает загрузки')]
global_log = global_log['№ площадки'].values
print(len(global_log))

In [ ]:
# submissions_path = "/content/drive/My Drive/москва/файлы_работ/"

# file_paths = []

# submissions_folder = submissions_path

# for root, directories, files in os.walk(submissions_folder):
#   for filename in files:
#     if '66798' in filename:
#     # if filename == 66798:
#       print(root, directories, filename)
#     file_path = os.path.join(root, filename)
#     file_paths.append(file_path)

# for f in file_paths:
#   print(f)

# response = service_drive.files(
#     q="name='66798' and mimeType='application/vnd.google-apps.folder' and '10GWqMWoVAVgxvdcFF3D_x4iqj3OeVoYl' in parents and trashed=false",
#     includeItemsFromAllDrives=True,
#     supportsAllDrives=True
# ).execute()

id = 66798
pdf_parent_folder = '10GWqMWoVAVgxvdcFF3D_x4iqj3OeVoYl'

query = f"name contains '{id}' and trashed=false and parents='{pdf_parent_folder}'and mimeType='application/vnd.google-apps.folder'"
results = service_drive.files().list(q=query, fields='files(id, name, parents, trashed, createdTime)', orderBy='createdTime').execute()
print(len(results))
pdf = results.get('files', [])[0]
print(pdf)
# for p in pdf:
#   print(p)

1
{'parents': ['10GWqMWoVAVgxvdcFF3D_x4iqj3OeVoYl'], 'id': '11P19QHki6I-uYf-WVb4z_0b5HHF-sCoJ', 'name': '66798', 'trashed': False, 'createdTime': '2023-04-19T19:24:11.422Z'}


In [ ]:
# site = '66900'
# spreadsheet_path = '1e3vFj5adphGAvv3JROtrwz9jm1yF3SQi'

site = 'Копия 66870'
spreadsheet_path = '1sqF9JHONRT_OUCvWV5ZlCxCMODOpaCNU'

query = f"name contains '{site}' and trashed=false and parents='{spreadsheet_path}'"
print(query)
results = service_drive.files().list(q=query, fields='files(id, name, parents, trashed, createdTime)', orderBy='createdTime').execute()
table_id = results.get('files', [])[0]['id']
gsheet = gclient.open_by_key(table_id)
df = pd.DataFrame(gsheet.get_worksheet(0).get_all_records())
df[["Фамилия", "Имя", "Кодовое слово"]] = df[["Фамилия", "Имя", "Кодовое слово"]].fillna(value=codeword)
df[["Фамилия", "Имя", "Кодовое слово"]] = df[["Фамилия", "Имя", "Кодовое слово"]].replace('', codeword, regex=True)

df['Пункт. ошибки'] = df['Пункт. ошибки'].astype(str)
df['Орф. ошибки'] = df['Орф. ошибки'].astype(str)



df['Орф. ошибки'] = df['Орф. ошибки'].apply(lambda x: x.strip())
df['Пункт. ошибки'] = df['Пункт. ошибки'].apply(lambda x: x.strip())

if df['Пункт. ошибки'].isnull().values.any()==True:
  print('punct, nan')

if df['Орф. ошибки'].isnull().values.any()==True:
  print('orth, nan')

# if df['Орф. ошибки'].astype(str).str.contains('').any()==True:
#   print('orth, empty')

# if df['Пункт. ошибки'].astype(str).str.contains('').any()==True:
#   print('punct, empty')

# if (df['Орф. ошибки'].eq('')).any() == True:
#   print('orty, empty')

# if (df['Пункт. ошибки'].eq('')).any() == True:
#   print('punct, empty')


cond_1 = df['Пункт. ошибки'].isnull().values.any()
cond_2 = df['Орф. ошибки'].isnull().values.any()
cond_3 = (df['Орф. ошибки'].eq('')).any()
cond_4 = (df['Пункт. ошибки'].eq('')).any()

if (cond_1 == True) or (cond_2 == True) or (cond_3 == True) or (cond_4 == True):
  print('missing')

name contains 'Копия 66870' and trashed=false and parents='1sqF9JHONRT_OUCvWV5ZlCxCMODOpaCNU'
missing


In [ ]:
# for s in df['Орф. ошибки'].astype(str):
#   print(s)

print(df['Орф. ошибки'].astype(str))

0       0
1       0
2       0
3       1
4     999
5       1
6       0
7     999
8     999
9       7
10      1
11     10
12      6
13    999
14      3
15      2
16    999
17      1
18     10
19      9
Name: Орф. ошибки, dtype: object


In [ ]:
file_paths = []

for root, directories, files in os.walk('/content/drive/My Drive/test_folder/'):
  for filename in files:
    file_path = os.path.join(root, filename)
    file_paths.append(file_path)

with ZipFile("/content/drive/My Drive/москва/site.zip", 'w') as zip:
  for file in file_paths:
    # print(file.split('/')[-1])
    zip.write(file, file.split('/')[-1])

# "directory/QuickSort.class", "QuickSort.class"

In [ ]:

url = urlopen("https://drive.google.com/drive/folders/1NAFLKyg8Vlf2xZ24He_VSHOf7xBxeBwN")
zipfile = "/content/drive/My Drive/москва/site.zip"

with ZipFile(zipfile, 'w') as zip:
         zip.write(StringIO(url))

# zipfile = ZipFile(StringIO(url))

# zip.write(input_zip)

TypeError: ignored

In [ ]:
f = google_drive.CreateFile({'title': 'test_site.zip'})
f.SetContentFile("/content/drive/My Drive/test_folder/66720")
f.Upload()

IsADirectoryError: ignored

In [ ]:
file1 = google_drive.CreateFile({'title': 'Hello.txt'})
file1.SetContentFile("/content/drive/My Drive/test_folder/66720")
file1.Upload()

IsADirectoryError: ignored

In [ ]:
file_path = "/content/drive/My Drive/test_folder/66720"
file1 = google_drive.CreateFile()
file1.SetContentFile(file_path)
file1.Upload()

IsADirectoryError: ignored

In [ ]:
zf = ZipFile("/content/drive/My Drive/test_folder/66720")

IsADirectoryError: ignored

In [ ]:
gsheet = gclient.open_by_key('1hQk6O9qUiYxD3uTymIPo2PspMfh1jryzjHzN0-wTik4')
df = pd.DataFrame(gsheet.get_worksheet(0).get_all_records())
df[["Фамилия", "Имя", "Кодовое слово"]] = df[["Фамилия", "Имя", "Кодовое слово"]].fillna(value='интеллигент')
df[["Фамилия", "Имя", "Кодовое слово"]] = df[["Фамилия", "Имя", "Кодовое слово"]].replace('', 'интеллигент', regex=True)
df

,Id работы,Автор,Фамилия,Имя,Кодовое слово,Результат,Орф. ошибки,Пункт. ошибки,Скан работы
0,66720_1,,Бородина-аксёнова,"elena, helena",Неуч/123,,9,7,https://drive.google.com/file/d/1s-E1I44OZyAtk...
1,66720_2,,Бородина-аксёнова,"elena, helena",Неуч/123,,9,7,https://drive.google.com/file/d/1ruI3xuFtxIXI7...
2,66720_3,,Богданова,Татьяна ивановна,sophia~567/,,0,3,https://drive.google.com/file/d/1rftyFpuSUbqv1...
3,66720_4,,Богданова,Татьяна ивановна,sophia~567/,,0,3,https://drive.google.com/file/d/1rVvShIACi7G5d...
4,66720_5,,интеллигент,Мария,Маша,,4,7,https://drive.google.com/file/d/1rNhGOM_4yLJYu...
5,66720_6,,интеллигент,Мария,Маша,,4,7,https://drive.google.com/file/d/1spARUVQbCTJwM...
6,66720_7,,Мацюк,Анна,интеллигент,,10,5,https://drive.google.com/file/d/1sQqoR7tIFIPGj...
7,66720_8,,Мацюк,Анна,интеллигент,,10,5,https://drive.google.com/file/d/1sLcZuBxWPEL1E...
8,66720_9,,"boyko, bojko",Анна,"кошка, кожка, кожна",,6,9,https://drive.google.com/file/d/1s9K8fyhdt73H9...
9,66720_10,,"Пересылкина, пересыпкина","Татьяна, татиана",Мария,,2,2,https://drive.google.com/file/d/1tfVNYJcPR9n2i...


In [ ]:
df.isnull().values.any()

False

In [ ]:
df_vals = []

In [ ]:
folder_1 = "/content/drive/My Drive/москва/файлы_работ/"
pdf_folder = '10GWqMWoVAVgxvdcFF3D_x4iqj3OeVoYl'


df[['Фамилия', 'Имя', 'Кодовое слово', 'Орф. ошибки', 'Пункт. ошибки']] = df[['Фамилия', 'Имя', 'Кодовое слово', 'Орф. ошибки', 'Пункт. ошибки']].fillna('')
df[['Фамилия', 'Имя', 'Кодовое слово']] = df[['Фамилия', 'Имя', 'Кодовое слово']].apply(lambda x: x.astype(str).str.lower())
duplicate = df[df.duplicated(subset=['Фамилия', 'Имя', 'Кодовое слово'], keep=False)]
grouped = duplicate.groupby(['Фамилия', 'Имя', 'Кодовое слово'], sort=False)

for name, group in grouped:
  merger = PdfMerger()
  parts = []
  for filename in group['Id работы'].values:
    path_1 = folder_1 + filename + '.pdf'
    merger.append(path_1)
    split_filename = filename.split('_')
    if split_filename[0] not in parts:
      parts.append(split_filename[0])
    parts.append(split_filename[1])
  new_filename = '_'.join(parts) + '.pdf'
  # print(new_filename)
  path_2 = folder_1 + new_filename
  merger.write(path_2)
  # time.sleep(15)
  # query = f"name contains '{new_filename}' and trashed=false and parents='{pdf_folder}'"
  # results = service_drive.files().list(q=query, fields='files(id, name, parents, trashed, createdTime)', orderBy='createdTime').execute()
  # pdf = results.get('files', [])[0]
  # print(pdf['name'], pdf['id'])
  new_row = group.index[0], new_filename.strip('.pdf'), group['Фамилия'].values[0], group['Имя'].values[0], group['Кодовое слово'].values[0], group['Орф. ошибки'].values[0], group['Пункт. ошибки'].values[0]

  df_vals.append(new_row)








In [ ]:
duplicates_df = pd.DataFrame(df_vals, columns = ['index', 'Id работы', 'Фамилия', 'Имя', 'Кодовое слово', 'Орф. ошибки', 'Пункт. ошибки'])
duplicates_df

,index,Id работы,Фамилия,Имя,Кодовое слово,Орф. ошибки,Пункт. ошибки
0,0,66720_1_2,бородина-аксёнова,"elena, helena",неуч/123,9,7
1,2,66720_3_4,богданова,татьяна ивановна,sophia~567/,0,3
2,4,66720_5_6,интеллигент,мария,маша,4,7
3,6,66720_7_8,мацюк,анна,интеллигент,10,5


In [ ]:
print(df.shape)
print(duplicate.shape)

rest_df = df.loc[~df.index.isin(duplicate.index)]
# rest['index'] = rest.index
# rest = rest.reset_index(inplace=True)
rest_df['index'] = rest_df.index
rest_df = rest_df.iloc[:, [9, 0,2,3, 4, 6, 7]]
rest_df.columns = duplicates_df.columns
# rest_df.columns =  ['Id работы', 'Фамилия', 'Имя', 'Кодовое слово', 'Орф. ошибки', 'Пункт. ошибки', 'index']
# print(rest.shape)
# print(rest.columns)

rest_df

(31, 9)
(8, 9)


,index,Id работы,Фамилия,Имя,Кодовое слово,Орф. ошибки,Пункт. ошибки
8,8,66720_9,"boyko, bojko",анна,"кошка, кожка, кожна",6,9
9,9,66720_10,"пересылкина, пересыпкина","татьяна, татиана",мария,2,2
10,10,66720_11,"черепенникова, черепепникова",дарья,медведь,2,2
11,11,66720_12,борисова,мария,школа,9,4
12,12,66720_13,прокина,даря,варна,8,7
13,13,66720_14,пащенко,наталья,фикрайт,1,2
14,14,66720_15,громова,мария,колесо,0,4
15,15,66720_16,кондратьев,максим,золото,3,7
16,16,66720_17,безменова,татьяна,василиса,1,1
17,17,66720_18,арсланова,регина,роберт,11,15


In [ ]:
new_df = pd.concat([duplicates_df, rest_df])
new_df.sort_values(by=['index'], inplace=True)
new_df.reset_index(inplace=True, drop=True)
new_df = new_df.iloc[:, [1, 2, 3, 4, 5, 6]]
new_df.columns = ['id', 'surname', 'name', 'code', 'orth', 'punct']
print(new_df.shape)
new_df

(27, 6)


,id,surname,name,code,orth,punct
0,66720_1_2,бородина-аксёнова,"elena, helena",неуч/123,9,7
1,66720_3_4,богданова,татьяна ивановна,sophia~567/,0,3
2,66720_5_6,интеллигент,мария,маша,4,7
3,66720_7_8,мацюк,анна,интеллигент,10,5
4,66720_9,"boyko, bojko",анна,"кошка, кожка, кожна",6,9
5,66720_10,"пересылкина, пересыпкина","татьяна, татиана",мария,2,2
6,66720_11,"черепенникова, черепепникова",дарья,медведь,2,2
7,66720_12,борисова,мария,школа,9,4
8,66720_13,прокина,даря,варна,8,7
9,66720_14,пащенко,наталья,фикрайт,1,2


In [ ]:
new_df = new_df.assign(surname=new_df['surname'].str.split(',')).explode('surname')
new_df = new_df.assign(name=new_df['name'].str.split(',')).explode('name')
new_df = new_df.assign(code=new_df['code'].str.split(',')).explode('code')
for key, val in translit.items():
  new_df[["surname", "name", "code"]] = new_df[["surname", "name", "code"]].replace(key, val, regex=True)
for key, val in nums.items():
  new_df[["surname", "name", "code"]] = new_df[["surname", "name", "code"]].replace(key, val, regex=True)
for merger in mergers:
  new_df[["surname", "name", "code"]] = new_df[["surname", "name", "code"]].replace(merger, '', regex=True)
new_df['surname'] = new_df['surname'].str.replace('[^ёа-яЁА-Я]', '', regex=True)
new_df['name'] = new_df['name'].str.replace('[^ёа-яЁА-Я]', '', regex=True)
new_df['code'] = new_df['code'].str.replace('[^ёа-яЁА-Я]', '', regex=True)
new_df.reset_index(inplace=True, drop=True)
new_df["ФИО"] = new_df["surname"] + ' ' + new_df["name"]
new_df = new_df[['ФИО', 'orth', 'punct', 'code', 'id']]
new_df['id'] = new_df['id'].astype(str) + '.pdf'
new_df.columns = ['ФИО', 'Орфография', 'Пунктуация', 'Кодовое слово', 'Скан']
print(new_df.shape)
new_df

(37, 5)


,ФИО,Орфография,Пунктуация,Кодовое слово,Скан
0,бородинааксёнова елена,9,7,неучодиндватри,66720_1_2.pdf
1,бородинааксёнова хелена,9,7,неучодиндватри,66720_1_2.pdf
2,богданова татьянаивановна,0,3,сопхиапятьшестьсемь,66720_3_4.pdf
3,интеллигент мария,4,7,маша,66720_5_6.pdf
4,мацюк анна,10,5,интеллигент,66720_7_8.pdf
5,боыко анна,6,9,кошка,66720_9.pdf
6,боыко анна,6,9,кожка,66720_9.pdf
7,боыко анна,6,9,кожна,66720_9.pdf
8,бойко анна,6,9,кошка,66720_9.pdf
9,бойко анна,6,9,кожка,66720_9.pdf


In [ ]:
child_folder = google_drive.CreateFile({'title': '66720', 'parents':[{'id':'1i7osa53I5Qh0sw6kZdDRrVck8cAS_aVX'}], 'mimeType': "application/vnd.google-apps.folder"})
child_folder.Upload()

In [ ]:
folder_name = '66720'
parent_folder = '1i7osa53I5Qh0sw6kZdDRrVck8cAS_aVX'
query = f"name contains '{folder_name}' and trashed=false and parents='{parent_folder}'"
copy_folder = service_drive.files().list(q=query, fields='files(id, name, parents, trashed, createdTime)', orderBy='createdTime').execute()
copy_folder = copy_folder.get('files', [])[0]['id']
print(copy_folder)

1NAFLKyg8Vlf2xZ24He_VSHOf7xBxeBwN


In [ ]:
pdf_parent_folder = '10GWqMWoVAVgxvdcFF3D_x4iqj3OeVoYl'
for id in new_df['id'].values:
  print(id)
  query = f"name contains '{id}' and trashed=false and parents='{pdf_parent_folder}'"
  results = service_drive.files().list(q=query, fields='files(id, name, parents, trashed, createdTime)', orderBy='createdTime').execute()
  pdf = results.get('files', [])[0]
  # pdf = results.get('files', [])
  # print(pdf['name'], pdf['id'])
  service_drive.files().copy(fileId=pdf['id'], body={"parents": [copy_folder], "mimeType":"application/pdf"} ).execute()


66720_1_2
66720_1_2.pdf 1-2El_KzHIFCyREtgVvV4u3dNNrNNpPCs
66720_1_2
66720_1_2.pdf 1-2El_KzHIFCyREtgVvV4u3dNNrNNpPCs
66720_3_4
66720_3_4.pdf 1-ETe3paOmWiD38gxSmMDI2nyx2chQTwc
66720_5_6
66720_5_6.pdf 1-Tk1eF91U_--7OX6EL3BarnXEU36MfOa
66720_7_8
66720_7_8.pdf 1-RBhQSsBT7wvVpsVbvmt6v8YCQtovApW
66720_9
66720_9.pdf 1s9K8fyhdt73H9woyZzmywlEdnWt_Se7T
66720_9
66720_9.pdf 1s9K8fyhdt73H9woyZzmywlEdnWt_Se7T
66720_9
66720_9.pdf 1s9K8fyhdt73H9woyZzmywlEdnWt_Se7T
66720_9
66720_9.pdf 1s9K8fyhdt73H9woyZzmywlEdnWt_Se7T
66720_9
66720_9.pdf 1s9K8fyhdt73H9woyZzmywlEdnWt_Se7T
66720_9
66720_9.pdf 1s9K8fyhdt73H9woyZzmywlEdnWt_Se7T
66720_10
66720_10.pdf 1tfVNYJcPR9n2ioPxAbHM5PpkBXxVtbJT
66720_10
66720_10.pdf 1tfVNYJcPR9n2ioPxAbHM5PpkBXxVtbJT
66720_10
66720_10.pdf 1tfVNYJcPR9n2ioPxAbHM5PpkBXxVtbJT
66720_10
66720_10.pdf 1tfVNYJcPR9n2ioPxAbHM5PpkBXxVtbJT
66720_11
66720_11.pdf 1tS4zy8g-fu2WD4LGRg8eaWi6ooMveoDp
66720_11
66720_11.pdf 1tS4zy8g-fu2WD4LGRg8eaWi6ooMveoDp
66720_12
66720_12.pdf 1tI32hNBH6uEqJEmmrP59zm0itOC

In [ ]:
# new_df = new_df[['Фамилия', 'Имя', 'Кодовое слово', 'Орф. ошибки', 'Пункт. ошибки']]
# new_df

In [ ]:
output_file = "/content/drive/My Drive/москва/66720.csv"
new_df.to_csv(output_file, sep=';', encoding='cp1251', index=False)

In [ ]:
with ZipFile('/content/drive/My Drive/66720.zip', 'w') as zip:
  zip.write('/content/drive/My Drive/test_folder/66720')
